In [ ]:
#@title
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from PIL import Image
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [ ]:

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets download niharika41298/yoga-poses-dataset

 98% 282M/288M [00:07<00:00, 51.2MB/s]
100% 288M/288M [00:07<00:00, 39.7MB/s]


In [ ]:
#@title
! unzip yoga-poses-dataset.zip

Archive:  yoga-poses-dataset.zip
  inflating: DATASET/TEST/downdog/00000000.jpg  
  inflating: DATASET/TEST/downdog/00000001.JPG  
  inflating: DATASET/TEST/downdog/00000002.jpg  
  inflating: DATASET/TEST/downdog/00000003.jpg  
  inflating: DATASET/TEST/downdog/00000004.jpg  
  inflating: DATASET/TEST/downdog/00000005.jpg  
  inflating: DATASET/TEST/downdog/00000006.jpg  
  inflating: DATASET/TEST/downdog/00000007.jpg  
  inflating: DATASET/TEST/downdog/00000008.jpg  
  inflating: DATASET/TEST/downdog/00000009.jpg  
  inflating: DATASET/TEST/downdog/00000010.png  
  inflating: DATASET/TEST/downdog/00000011.jpg  
  inflating: DATASET/TEST/downdog/00000012.jpg  
  inflating: DATASET/TEST/downdog/00000013.jpg  
  inflating: DATASET/TEST/downdog/00000014.jpg  
  inflating: DATASET/TEST/downdog/00000016.jpg  
  inflating: DATASET/TEST/downdog/00000017.png  
  inflating: DATASET/TEST/downdog/00000018.jpg  
  inflating: DATASET/TEST/downdog/00000019.jpg  
  inflating: DATASET/TEST/downdog/00

In [ ]:
train_dir = '../content/DATASET/TRAIN'
test_dir = '../content/DATASET/TEST'

In [ ]:


train_datagen = ImageDataGenerator(width_shift_range= 0.1,
                                  horizontal_flip = True,
                                  rescale = 1./255,
                                  validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 validation_split = 0.2)



In [ ]:
train_generator =  train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (224,224),
                                                    color_mode = 'rgb',
                                                    class_mode = 'categorical',
                                                    batch_size = 16,
                                                    subset = 'training')
validation_generator  = test_datagen.flow_from_directory(directory = test_dir,
                                                  target_size = (224,224),
                                                  color_mode = 'rgb',
                                                  class_mode = 'categorical',
                                                  subset = 'validation')

Found 866 images belonging to 5 classes.
Found 92 images belonging to 5 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding = 'Same', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy'])
epochs = 50
batch_size = 16

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 112, 112, 64)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 56, 56, 128)      

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True



In [ ]:
history = model.fit(train_generator, epochs = epochs, validation_data = validation_generator, verbose=1)

Epoch 1/50
11/55 [=====>........................] - ETA: 17s - loss: 1.4462 - accuracy: 0.3889

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


55/55 [==============================] - 27s 499ms/step - loss: 1.4405 - accuracy: 0.4180 - val_loss: 1.1311 - val_accuracy: 0.5109
Epoch 2/50
55/55 [==============================] - 26s 482ms/step - loss: 1.3729 - accuracy: 0.4296 - val_loss: 1.1106 - val_accuracy: 0.5652
Epoch 3/50
55/55 [==============================] - 26s 476ms/step - loss: 1.2971 - accuracy: 0.4619 - val_loss: 1.0495 - val_accuracy: 0.6522
Epoch 4/50
55/55 [==============================] - 26s 474ms/step - loss: 1.2468 - accuracy: 0.4942 - val_loss: 0.9462 - val_accuracy: 0.6087
Epoch 5/50
55/55 [==============================] - 26s 479ms/step - loss: 1.1538 - accuracy: 0.5624 - val_loss: 0.8980 - val_accuracy: 0.6739
Epoch 6/50
55/55 [==============================] - 27s 500ms/step - loss: 1.0685 - accuracy: 0.6039 - val_loss: 0.7726 - val_accuracy: 0.6957
Epoch 7/50
55/55 [==============================] - 26s 477ms/step - loss: 1.0694 - accuracy: 0.5935 - val_loss: 0.8321 - val_accuracy: 0.7391
Epoch 8/50

KeyboardInterrupt: ignored

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

NameError: ignored

In [ ]:


train_loss, train_acc = model.evaluate(train_generator)
test_loss, test_acc   = model.evaluate(validation_generator)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_acc*100, test_acc*100))



 8/55 [===>..........................] - ETA: 17s - loss: 0.0515 - accuracy: 0.9922

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


3/3 [==============================] - 3s 919ms/step - loss: 0.6020 - accuracy: 0.8370
final train accuracy = 98.61 , validation accuracy = 83.70


In [ ]:

model.save('YogaNet_mode.h5')



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# copy it there
!cp YogaNet_model_1_1.h5 /content/drive/MyDrive

Mounted at /content/drive


In [ ]:
!cp YogaNet_model_1_1.h5 /content/drive/MyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
